In [1]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules

from modSAR.datasource import ChEMBLApiDataSource
from modSAR.cdk_utils import JavaCDKBridge

/opt/conda/lib/python3.6/site-packages/grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (/opt/conda/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py)', 'urllib3.util (/opt/conda/lib/python3.6/site-packages/urllib3/util/__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)


TODO DOCUMENTATION:

- Using ChEMBL API to download inhibitors
- Or prepare your own inhibitors
- Calculate molecular descriptors


# Acquiring Data from ChEMBL

The class `ChEMBLApiDataSource` interacts with the [ChEMBL webresource client API](https://github.com/chembl/chembl_webresource_client) to download bioactivities from ChEMBL. 

We must pass the ID of the target protein and the standard types we are interested in and the class will compile the compounds requested into a pandas DataFrame `bioactivities_df`:

In [2]:
chembl_data_source = ChEMBLApiDataSource(target_id='CHEMBL202', standard_types=['IC50', 'Ki'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In total, the API returned 1500+ bioactivities registered as hDHFR ligands [CHEMBL202](https://www.ebi.ac.uk/chembl/target/inspect/CHEMBL202):

In [3]:
chembl_data_source

ChEMBLApiDataSource object
  target_id: CHEMBL202
  bioactivities: 1573
  standard_types: ['Ki' 'IC50' 'Log 1/Ki app' 'IC50/[E]' 'Ratio IC50']

Here is a peak of the data returned from ChEMBL API:

In [4]:
chembl_data_source.bioactivities_df.head()

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,published_relation,published_type,published_units,published_value,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,40879,CHEMBL858267,Inhibition of human dihydrofolate reductase (D...,B,None,BAO_0000357,single protein format,CC1(C)N=C(N)N=C(N)N1c2cccc(Cl)c2,None,None,CHEMBL1128273,J. Med. Chem.,1995,None,CHEMBL7130,None,CHEMBL7130,7.03,False,=,Log 1/Ki,None,7.03,None,347068,=,1,True,=,None,Ki,nM,None,93.33,CHEMBL202,Homo sapiens,Dihydrofolate reductase,9606,None,None,Log 1/Ki,None,None,None,7.03
0,None,42137,CHEMBL858267,Inhibition of human dihydrofolate reductase (D...,B,None,BAO_0000357,single protein format,CC1(C)N=C(N)N=C(N)N1c2ccc(CCCCc3ccc(cc3Cl)S(=O...,None,None,CHEMBL1128273,J. Med. Chem.,1995,None,CHEMBL33697,None,CHEMBL33697,7.65,False,=,Log 1/Ki,None,7.65,None,347071,=,1,True,=,None,Ki,nM,None,22.39,CHEMBL202,Homo sapiens,Dihydrofolate reductase,9606,None,None,Log 1/Ki,None,None,None,7.65
0,None,42149,CHEMBL858267,Inhibition of human dihydrofolate reductase (D...,B,None,BAO_0000357,single protein format,CC1(C)N=C(N)N=C(N)N1c2cccc(OCC34CC5CC(CC(C5)C3...,None,None,CHEMBL1128273,J. Med. Chem.,1995,None,CHEMBL281618,None,CHEMBL281618,6.11,False,=,Log 1/Ki,None,6.11,None,347091,=,1,True,=,None,Ki,nM,None,776.25,CHEMBL202,Homo sapiens,Dihydrofolate reductase,9606,None,None,Log 1/Ki,None,None,None,6.11
0,None,45682,CHEMBL858267,Inhibition of human dihydrofolate reductase (D...,B,None,BAO_0000357,single protein format,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC,None,None,CHEMBL1128273,J. Med. Chem.,1995,None,CHEMBL22,TRIMETHOPRIM,CHEMBL22,6.71,False,=,Log 1/Ki,None,6.71,None,347106,=,1,True,=,None,Ki,nM,None,194.98,CHEMBL202,Homo sapiens,Dihydrofolate reductase,9606,None,None,Log 1/Ki,None,None,None,6.71
0,None,46931,CHEMBL666808,Inhibition of human dihydrofolate reductase (D...,B,None,BAO_0000357,single protein format,CC1(C)N=C(N)N=C(N)N1c2cccc(SCc3ccccc3)c2,None,None,CHEMBL1128273,J. Med. Chem.,1995,None,CHEMBL20975,None,CHEMBL20975,7.37,False,=,Log 1/Ki,None,7.37,None,347115,=,1,True,=,None,Ki,nM,None,42.66,CHEMBL202,Homo sapiens,Dihydrofolate reductase,9606,None,None,Log 1/Ki,None,None,None,7.37


The distribution of standard activity types in the data set:

In [5]:
chembl_data_source.bioactivities_df['standard_type'].value_counts()

IC50            1090
Ki               435
Log 1/Ki app      38
IC50/[E]           8
Ratio IC50         2
Name: standard_type, dtype: int64

# Preprocessing

Methods in `Preprocessing` class filters out invalid/more inaccurate entries for QSAR modelling. In summary, the preprocessing step for ChEMBL data involves:

- Select only valid entries as indicated by the column `data_validity_comment`
- Select only entries where relation is of type equality (e.g.: IC50 = 30nM), as indicated by column `relation`
- Handle duplicated entries
- Remove data marked as outliers

To obtain a preprocessed dataset, either use `chembl_data_source.get_qsar_dataset()` method or the function `build_qsar_dataset(chembl_data_source)`:


In [15]:
qsar_dataset = chembl_data_source.get_qsar_dataset()

Cleaning up JavaGateway
Compiling CDKBridge
Starting CDKBridge


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:25333)

Because of this preprocessing step, the resulting data set has fewer entries:

In [ ]:
preprocessed_df.shape

Number of duplicated entries that existed in the dataset and were converted to a single entry:

In [ ]:
type(chembl_data_source) is ChEMBLApiDataSource

In [ ]:
preprocessed_df['duplicated'].sum()

In [ ]:
import altair as alt

alt.renderers.enable('notebook')

chart = alt.Chart(preprocessed_df).mark_bar().encode(
    x=alt.X("median_pchembl_value", bin=True, title='Median pChEMBL value'),
    y='count()',
    tooltip=['count()'],
).properties(
    title='Activities distribution'
)

chart

![Activities distribution](fig/notebook01/fig01_activities_distribution.png)

# Molecular Descriptors

After removing invalid and handling duplicated entries, we use Chemistry Development Kit ([CDK v2.2](https://github.com/cdk/cdk)) to calculate molecular descriptors on the resulting dataset.

In order to run CDK in Python, we need to establish a bridge with the Java programming language:

In [7]:
java_cdk_bridge = JavaCDKBridge()
java_cdk_bridge.start_cdk_java_bridge()

CDK Bridge process running


Now, we can instantiate classes and manipulate CDK Java objects from within Python:

In [11]:
gateway = java_cdk_bridge.gateway
cdk = gateway.jvm.org.openscience.cdk

builder = cdk.DefaultChemObjectBuilder.getInstance()
smiles_parser = cdk.smiles.SmilesParser(builder)
builder

JavaObject id=o2

In [ ]:
from py4j.java_gateway import JavaGateway, GatewayParameters
JavaGateway(gateway_parameters=GatewayParameters(auto_convert=True))

**DESCRIPTORS LIST**

For convenience, we saved a list of all molecular descriptors classes in CDK in the `descriptors_list.csv`. 

In [ ]:
descriptors_list = pd.read_csv('/mnt/data/descriptors_list.csv')

def remove_prefix(java_class_name):
    return java_class_name.replace('org.openscience.', '') + '()'

descriptors_list['object_invocation'] = descriptors_list['descriptorClass'].apply(remove_prefix)
descriptors_list

We can calculate and combine all molecular descriptors in a DataFrame: 

In [ ]:
import py4j

def calculate_all_descriptors(data, descriptors_df):
    mol_container = smiles_parser.parseSmiles(data['canonical_smiles'])
    dict_descriptors = {}
    for i in range(descriptors_df.shape[0]):
        descriptor = eval(descriptors_df.iloc[i]['object_invocation'])
        descriptor.initialise(builder)
        descriptor_names = [desc_name for desc_name in descriptor.getDescriptorNames()]
        try:
            descriptor_values = descriptor.calculate(mol_container).getValue().toString().split(',')
        except Exception as e:
            print(e)
        
        dict_descriptors.update({descriptor_names[j]: descriptor_values[j] 
                                 for j in range(len(descriptor_names))})
    
    result_df = pd.DataFrame(dict_descriptors, index=[data['parent_molecule_chembl_id']])
    return result_df

pd.concat([
    calculate_all_descriptors(preprocessed_df.iloc[1], descriptors_list),
    calculate_all_descriptors(preprocessed_df.iloc[0], descriptors_list)], axis=0)

\#TODO: Errors

In [ ]:
smiles_parser.parseSmiles(preprocessed_df.iloc[351]['canonical_smiles'])

In [ ]:
preprocessed_df.head(10).apply(calculate_all_descriptors, axis=1, descriptors_df=descriptors_list)

In [ ]:
py4j.java_gateway.get_field(descriptor, 'CHECK_RING_SYSTEM')

In [ ]:
[f for f in descriptor.getParameters()]

In [ ]:
gateway.jvm.java.lang.Object()

In [ ]:
descriptor.setParameters(['false'])

In [ ]:
descriptor = cdk.qsar.descriptors.molecular.LongestAliphaticChainDescriptor()
mol_container = smiles_parser.parseSmiles(preprocessed_df.iloc[2]['canonical_smiles'])
descriptor.calculate(mol_container).getValue().toString().split(',')

In [ ]:
m = Chem.MolFromSmiles(preprocessed_df.iloc[3]['canonical_smiles'])
m

In [ ]:
AllChem.CalcKappa1(m)

In [ ]:
result_df.columns.tolist()